<font size="6">Calculate GHG Intensity per GDP, PPP </font>

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

<font size="4">Load Environment Variables</font>

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [5]:
# Show available schemas to ensure trino connection is set correctly
ingest_catalog = 'osc_datacommons_dev'
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)
('wri_gppd',)


In [6]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_unfccc_results'
src_table_1 = 'sf_unfccc_with_lulucf'
src_table_2 = 'sf_unfccc_without_lulucf'
src_table_3 = 'sf_wdi_gdp'
src_table_4 = 'sf_primap_hist_emissions'
src_country_table = 'sf_unfccc_countries'

In [7]:
########################
essd_schema = 'mdt_sandbox'
essd_src_table = 'gwp100_data'
#essd_src_table = 'ghg_data' 

In [8]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [9]:
from openscm_units import unit_registry
#PintType.ureg = unit_registry
ureg = unit_registry
Q_ = ureg.Quantity
ureg.define("CO2e=CO2=CO2eq=CO2_eq")
ureg.define("USD=[currency]=$")

In [10]:
from pint import UnitRegistry, set_application_registry
set_application_registry(ureg)

In [11]:
import pandas as pd
from functools import reduce
import pandas as pd
import pint
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from pint_pandas.pint_array import is_pint_type

In [12]:
### ESSD data is not needed at the moment (maybe in the future)

'''
sql=f"""
select iso as country_iso_code,year(year) as validity_date, co2/1000 as value,t.* from {ingest_catalog}.{essd_schema}.{essd_src_table} t """ 
df_essd = pd.read_sql(sql, engine)
#df1["units"] = "kt"
df_essd = df_essd.convert_dtypes()
df_essd[df_essd['country_iso_code'] == 'ZWE'][df_essd['validity_date'] ==2020]
rename_columns = {'value':'total'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df_essd = df_essd.groupby(['country_iso_code','validity_date'],as_index=False).agg(agg_columns).rename(columns=rename_columns)
df_essd
'''



#df_essd.info(verbose=True)

'\nsql=f"""\nselect iso as country_iso_code,year(year) as validity_date, co2/1000 as value,t.* from {ingest_catalog}.{essd_schema}.{essd_src_table} t """ \ndf_essd = pd.read_sql(sql, engine)\n#df1["units"] = "kt"\ndf_essd = df_essd.convert_dtypes()\ndf_essd[df_essd[\'country_iso_code\'] == \'ZWE\'][df_essd[\'validity_date\'] ==2020]\nrename_columns = {\'value\':\'total\'}\nagg_columns = { \'value\' : \'sum\'}\ncolumns_order = [\'attribute\',\'industry_code\',\'country_iso_code\',\'validity_date\',\'total\',\'total_units\']\ndf_essd = df_essd.groupby([\'country_iso_code\',\'validity_date\'],as_index=False).agg(agg_columns).rename(columns=rename_columns)\ndf_essd\n'

In [15]:
sql=f"""
select country_iso_code,country_name,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
#df1["units"] = "kt"
df1 = requantify_df(df1).convert_dtypes()
df1
df1.info(verbose=True)
sql=f"""
select t.country_iso_code,country_name,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_2} t, {ingest_catalog}.{ingest_schema}.{src_country_table} c
where t.country_iso_code= c.country_iso_code and c.annex1_flag='Y'"""
df2 = pd.read_sql(sql, engine)
#df2 = requantify_df(df2).convert_dtypes()
#df2
# gdp
sql=f"""
select country_iso_code,validity_date,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} where attribute='GDP (current US$)'"""
df3 = pd.read_sql(sql, engine)
df3 = requantify_df(df3).convert_dtypes()
# gdp ppp
sql=f"""
select country_iso_code,validity_date,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} where attribute='GDP, PPP (current international $)'"""
df4 = pd.read_sql(sql, engine)
df4 = requantify_df(df4).convert_dtypes()

# PRIMAP data

sql=f"""
select country_iso_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_4} where attribute='CO2' and validity_date >=1960 """ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212 entries, 0 to 2211
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   country_iso_code  2212 non-null   string         
 1   country_name      2212 non-null   string         
 2   validity_date     2212 non-null   Int64          
 3   attribute         2212 non-null   string         
 4   value             2212 non-null   pint[CO2e * kt]
dtypes: Int64(1), pint[CO2e * kt](1), string(3)
memory usage: 88.7 KB


In [16]:
df_primap = pd.read_sql(sql, engine)
#df_primap['value_units'] = 'kt CO2'
#df_primap = requantify_df(df_primap).convert_dtypes()
# convert to CO2 units to t
#df_primap['value'] = df_primap['value'].pint.to("t CO2e")
df_primap


#df2 = pd.merge(df2,df_essd,on=['country_iso_code','validity_date'],how='outer')  
#df2.rename(columns={"value": "ghg_total_without_lulucf", "total": "ghg_total_without_lulucf_essd"},inplace=True)
df2 = pd.merge(df2,df_primap,on=['country_iso_code','validity_date'],how='outer')  
df2.rename(columns={"value_units_x":"value_units","value_x": "ghg_total_without_lulucf_unfccc","value_y": "ghg_total_without_lulucf_primap"},inplace=True)
df2=df2.convert_dtypes()
df2.info(verbose=True)
df_primap.info(verbose=True)
df2



<class 'pandas.core.frame.DataFrame'>
Int64Index: 12840 entries, 0 to 12839
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   country_iso_code                 12840 non-null  string 
 1   country_name                     1290 non-null   string 
 2   validity_date                    12840 non-null  Int64  
 3   attribute_x                      1290 non-null   string 
 4   ghg_total_without_lulucf_unfccc  1290 non-null   Float64
 5   value_units                      1290 non-null   string 
 6   attribute_y                      12840 non-null  string 
 7   ghg_total_without_lulucf_primap  12813 non-null  Float64
 8   value_units_y                    12840 non-null  string 
dtypes: Float64(2), Int64(1), string(6)
memory usage: 1.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12840 entries, 0 to 12839
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dt

,country_iso_code,country_name,validity_date,attribute_x,ghg_total_without_lulucf_unfccc,value_units,attribute_y,ghg_total_without_lulucf_primap,value_units_y
0,JPN,Japan,1990,"Time Series - GHG total without LULUCF, in kt ...",1269015.302881,kt CO2e,CO2,1160000.0,Gg CO2 / yr
1,JPN,Japan,1991,"Time Series - GHG total without LULUCF, in kt ...",1283386.635185,kt CO2e,CO2,1170000.0,Gg CO2 / yr
2,JPN,Japan,1992,"Time Series - GHG total without LULUCF, in kt ...",1295604.146623,kt CO2e,CO2,1180000.0,Gg CO2 / yr
3,JPN,Japan,1993,"Time Series - GHG total without LULUCF, in kt ...",1291284.868338,kt CO2e,CO2,1170000.0,Gg CO2 / yr
4,JPN,Japan,1994,"Time Series - GHG total without LULUCF, in kt ...",1352190.749204,kt CO2e,CO2,1230000.0,Gg CO2 / yr
...,...,...,...,...,...,...,...,...,...
12835,DOM,<NA>,2015,<NA>,<NA>,<NA>,CO2,14700.0,Gg CO2 / yr
12836,DOM,<NA>,2016,<NA>,<NA>,<NA>,CO2,15300.0,Gg CO2 / yr
12837,DOM,<NA>,2017,<NA>,<NA>,<NA>,CO2,15200.0,Gg CO2 / yr
12838,DOM,<NA>,2018,<NA>,<NA>,<NA>,CO2,15600.0,Gg CO2 / yr


In [17]:
import numpy as np
df2['scope1_excl_source'] = np.where(df2.ghg_total_without_lulucf_unfccc.notnull(),
         'UNFCCC',
         np.where(df2.ghg_total_without_lulucf_primap.notnull(),
                  'PRIMAP','UNFCCC'))
#df2['scope1_source'] = df2['ghg_total_without_lulucf'].apply(lambda x: 'UNFCCC' if x.isnull() == True else 'PRIMAP')
df2["value"] = df2["ghg_total_without_lulucf_unfccc"].fillna(df2["ghg_total_without_lulucf_primap"])
df2.drop(columns=['attribute_x','attribute_y',"value_units_y","ghg_total_without_lulucf_unfccc","ghg_total_without_lulucf_primap"],inplace=True)

columns_order = ['country_iso_code','country_name','validity_date','value','value_units','scope1_excl_source']
df2 = df2.reindex(columns=columns_order).convert_dtypes()
df2.info(verbose=True)
##df2[df2["scope1_excl_source"] =='PRIMAP']
#df2 = requantify_df(df2)

#df2

#df2.sort_values(by=['country_iso_code','validity_date'], ascending=False).to_csv("tmp_scope1_waterfall.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12840 entries, 0 to 12839
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country_iso_code    12840 non-null  string 
 1   country_name        1290 non-null   string 
 2   validity_date       12840 non-null  Int64  
 3   value               12813 non-null  Float64
 4   value_units         1290 non-null   string 
 5   scope1_excl_source  12840 non-null  string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 727.3 KB


Read the source tables into dataframes  

<font size="4">Merge the source dataframes in a single dataframe and calculate ghg intensity values</font>

In [18]:
df2 = df2.assign(value=np.where(df2.value.isnull(), 0, df2.value))
df2 = requantify_df(df2)
df2

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,country_iso_code,country_name,validity_date,value,scope1_excl_source
0,JPN,Japan,1990,1269015.302880783,UNFCCC
1,JPN,Japan,1991,1283386.6351854408,UNFCCC
2,JPN,Japan,1992,1295604.146622561,UNFCCC
3,JPN,Japan,1993,1291284.8683383777,UNFCCC
4,JPN,Japan,1994,1352190.7492043055,UNFCCC
...,...,...,...,...,...
12835,DOM,<NA>,2015,14700.0,PRIMAP
12836,DOM,<NA>,2016,15300.0,PRIMAP
12837,DOM,<NA>,2017,15200.0,PRIMAP
12838,DOM,<NA>,2018,15600.0,PRIMAP


In [19]:

df_result = pd.merge(df1,df2,on=['country_iso_code','validity_date'],how='outer')  
df_result=df_result.convert_dtypes()
df_result.drop(columns=['country_name_y','attribute'],inplace=True)
df_result.rename(columns={"country_name_x":"country_name","value_x": "ghg_total_with_lulucf","value_y": "ghg_total_without_lulucf"},inplace=True)

###

df_result = pd.merge(df_result,df3,on=['country_iso_code','validity_date'],how="outer")  
df_result.rename(columns={"value":"gdp"},inplace=True)
###
df_result = pd.merge(df_result,df4,on=['country_iso_code','validity_date'],how="outer")  
df_result.rename(columns={"value":"gdp_ppp"},inplace=True)



df_result["ghg_intensity_with_lulucf_per_gdp"] = df_result["ghg_total_with_lulucf"]/df_result["gdp_ppp"]*1000000
df_result["ghg_intensity_without_lulucf_per_gdp"] = df_result["ghg_total_without_lulucf"]/df_result["gdp_ppp"]*1000000
df_result.info(verbose=True)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 16186 entries, 0 to 16185
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype                
---  ------                                --------------  -----                
 0   country_iso_code                      16186 non-null  string               
 1   country_name                          2212 non-null   string               
 2   validity_date                         16186 non-null  Int64                
 3   ghg_total_with_lulucf                 2212 non-null   pint[CO2e * kt]      
 4   ghg_total_without_lulucf              12840 non-null  pint[CO2e * kt]      
 5   scope1_excl_source                    12840 non-null  string               
 6   gdp                                   12854 non-null  pint[USD]            
 7   gdp_ppp                               7210 non-null   pint[USD]            
 8   ghg_intensity_with_lulucf_per_gdp     2045 non-null   pint[CO2e * kt / USD]


In [20]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [21]:
df1 = dequantify_df(df1)
df2 = dequantify_df(df2)
df3 = dequantify_df(df3)
df_result = dequantify_df(df_result)
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16186 entries, 0 to 16185
Data columns (total 16 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   country_iso_code                            16186 non-null  string 
 1   country_name                                2212 non-null   string 
 2   validity_date                               16186 non-null  Int64  
 3   ghg_total_with_lulucf                       2212 non-null   Float64
 4   ghg_total_with_lulucf_units                 16186 non-null  string 
 5   ghg_total_without_lulucf                    12840 non-null  Float64
 6   ghg_total_without_lulucf_units              16186 non-null  string 
 7   scope1_excl_source                          12840 non-null  string 
 8   gdp                                         12854 non-null  Float64
 9   gdp_units                                   16186 non-null  string 
 10  gdp_ppp   

In [22]:

# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
CREATE SCHEMA if not exists {ingest_catalog}.{ingest_schema}
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )
"""
print(sql)
qres = engine.execute(sql)
#print(qres.fetchall())


CREATE SCHEMA if not exists osc_datacommons_dev.pcaf_sovereign_footprint
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )



Run these in a notebook cell if you need to install onto your nb env

<font size="5">Save the results in Trino</font>


In [23]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
#df_result=df_result.convert_dtypes()
#df_result.info()
columnschema = osc.create_table_schema_pairs(df_result) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
#print(qres.fetchall())




drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_results



In [24]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
#print(qres.fetchall())


create table if not exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_results(
    country_iso_code varchar,
    country_name varchar,
    validity_date bigint,
    ghg_total_with_lulucf double,
    ghg_total_with_lulucf_units varchar,
    ghg_total_without_lulucf double,
    ghg_total_without_lulucf_units varchar,
    scope1_excl_source varchar,
    gdp double,
    gdp_units varchar,
    gdp_ppp double,
    gdp_ppp_units varchar,
    ghg_intensity_with_lulucf_per_gdp double,
    ghg_intensity_with_lulucf_per_gdp_units varchar,
    ghg_intensity_without_lulucf_per_gdp double,
    ghg_intensity_without_lulucf_per_gdp_units varchar
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)



In [25]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
#print(qres.fetchall())

In [26]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,country_iso_code,country_name,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,gdp,gdp_units,gdp_ppp,gdp_ppp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units


In [27]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_unfccc_results"
inserting 1000 records
  ('DOM', 'Dominican Republic', 1990, 7084.93, 'CO2e * kt', 8890.0, 'CO2e * kt', 'PRIMAP', 7073675544.80847, 'USD', 26151683421.271, 'USD', 0.27091678519774864, 'CO2e * kt / USD', 0.3399398752574811, 'CO2e * kt / USD')
  ('DOM', 'Dominican Republic', 1994, 13937.575, 'CO2e * kt', 15100.0, 'CO2e * kt', 'PRIMAP', 14644711384.8202, 'USD', 35756703157.6101, 'USD', 0.3897891519406947, 'CO2e * kt / USD', 0.42229844103472014, 'CO2e * kt / USD')
  ('DOM', 'Dominican Republic', 1998, 4673.36, 'CO2e * kt', 16200.0, 'CO2e * kt', 'PRIMAP', 21672231760.6491, 'USD', 49800139033.4134, 'USD', 0.09384230828882645, 'CO2e * kt / USD', 0.32530029663432486, 'CO2e * kt / USD')
  ...
  ('ITA', 'Italy', 2014, 386901.0107928528, 'CO2e * kt', 427930.4118280704, 'CO2e * kt', 'UNFCCC', 2162009615996.54, 'USD', 2200255257934.9, 'USD', 0.17584369331583263, 'CO2e * kt / USD', 0.194491257

In [28]:
df_result.to_csv(ingest_table+".csv")
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)


,country_iso_code,country_name,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,gdp,gdp_units,gdp_ppp,gdp_ppp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units
0,GRC,Greece,1990,101181.552031,CO2e * kt,103289.457179,CO2e * kt,UNFCCC,9.789109e+10,USD,1.356630e+11,USD,0.745830,CO2e * kt / USD,0.761368,CO2e * kt / USD
1,GRC,Greece,1991,101039.148434,CO2e * kt,103344.245854,CO2e * kt,UNFCCC,1.051432e+11,USD,1.445988e+11,USD,0.698755,CO2e * kt / USD,0.714697,CO2e * kt / USD
2,GRC,Greece,1992,102199.579649,CO2e * kt,104588.354103,CO2e * kt,UNFCCC,1.162247e+11,USD,1.489292e+11,USD,0.686229,CO2e * kt / USD,0.702269,CO2e * kt / USD
3,GRC,Greece,1993,101339.660192,CO2e * kt,104188.433428,CO2e * kt,UNFCCC,1.088091e+11,USD,1.500199e+11,USD,0.675508,CO2e * kt / USD,0.694497,CO2e * kt / USD
4,GRC,Greece,1994,104388.516431,CO2e * kt,106975.033418,CO2e * kt,UNFCCC,1.166018e+11,USD,1.562880e+11,USD,0.667924,CO2e * kt / USD,0.684474,CO2e * kt / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16181,OED,None,1989,NaN,CO2e * kt,NaN,CO2e * kt,None,1.668684e+13,USD,NaN,USD,NaN,CO2e * kt / USD,NaN,CO2e * kt / USD
16182,OED,None,1992,NaN,CO2e * kt,NaN,CO2e * kt,None,2.139867e+13,USD,2.019058e+13,USD,NaN,CO2e * kt / USD,NaN,CO2e * kt / USD
16183,OED,None,2018,NaN,CO2e * kt,NaN,CO2e * kt,None,5.347575e+13,USD,6.122146e+13,USD,NaN,CO2e * kt / USD,NaN,CO2e * kt / USD
16184,OED,None,2010,NaN,CO2e * kt,NaN,CO2e * kt,None,4.520136e+13,USD,4.449183e+13,USD,NaN,CO2e * kt / USD,NaN,CO2e * kt / USD
